In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [45]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, Dropout, Activation
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__

'1.4.0'

In [4]:
tf.keras.__version__

'2.0.8-tf'

In [5]:
import imdb

In [6]:
imdb.data_dir = "data/IMDB/"

In [ ]:
imdb.maybe_download_and_extract()

Load the training- and test-sets.

In [7]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [8]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


In [9]:
data_text = x_train_text + x_test_text

In [10]:
x_train_text[1]

"The acting is pretty cheesy, but for the people in this area up in the 80s and are now Detroit area automotive engineers, this is a great movie. I even work with a Japanese supplier so that makes this movie even more funny.<br /><br />Jay Leno was showing his age last night on The Tonight Show! He looks pretty young here...17 years ago. The opening scene, with the drag race on what appears to be Woodward Ave was great.<br /><br />Leno also owns some bad a** cars now, it would e great to see a remake of this with his modern collection. I'm sure the blown Vette in the opening scene was his own car.<br /><br />Typical 80s movie. Watch it and enjoy. No computer generated crap!"

In [11]:
y_train[1]

1.0

In [12]:
num_words = 10000

In [13]:
??Tokenizer

In [14]:
tokenizer = Tokenizer(num_words=num_words)

In [15]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 10.7 s, sys: 52 ms, total: 10.8 s
Wall time: 10.8 s


In [16]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [17]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [18]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [19]:
x_train_text[1]

"The acting is pretty cheesy, but for the people in this area up in the 80s and are now Detroit area automotive engineers, this is a great movie. I even work with a Japanese supplier so that makes this movie even more funny.<br /><br />Jay Leno was showing his age last night on The Tonight Show! He looks pretty young here...17 years ago. The opening scene, with the drag race on what appears to be Woodward Ave was great.<br /><br />Leno also owns some bad a** cars now, it would e great to see a remake of this with his modern collection. I'm sure the blown Vette in the opening scene was his own car.<br /><br />Typical 80s movie. Watch it and enjoy. No computer generated crap!"

In [20]:
np.array(x_train_tokens[1])

array([   1,  113,    6,  180,  914,   18,   15,    1,   83,    8,   11,
       1598,   53,    8,    1, 2011,    2,   23,  146, 7540, 1598,   11,
          6,    3,   78,   17,   10,   57,  158,   16,    3,  936,   34,
         12,  162,   11,   17,   57,   51,  152,    7,    7, 3073, 7598,
         13,  761,   24,  599,  238,  316,   20,    1, 4352,  119,   28,
        285,  180,  186,  133, 2962,  153,  604,    1,  615,  129,   16,
          1, 2150, 1394,   20,   48,  733,    5,   26,   13,   78,    7,
          7, 7598,   81, 5663,   47,   74,    3, 1897,  146,    9,   58,
        932,   78,    5,   63,    3, 1044,    4,   11,   16,   24,  692,
       1520,  145,  247,    1, 2567,    8,    1,  615,  129,   13,   24,
        199,  520,    7,    7,  789, 2011,   17,  103,    9,    2,  354,
         54, 1293, 4761,  590])

In [21]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [22]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [23]:
np.mean(num_tokens)

221.27716

In [24]:
np.max(num_tokens)

2208

In [25]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

In [26]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94536

In [27]:
pad = 'pre'

In [28]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [29]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [30]:
x_train_pad.shape

(25000, 544)

In [31]:
x_test_pad.shape

(25000, 544)

In [32]:
np.array(x_train_tokens[1])

array([   1,  113,    6,  180,  914,   18,   15,    1,   83,    8,   11,
       1598,   53,    8,    1, 2011,    2,   23,  146, 7540, 1598,   11,
          6,    3,   78,   17,   10,   57,  158,   16,    3,  936,   34,
         12,  162,   11,   17,   57,   51,  152,    7,    7, 3073, 7598,
         13,  761,   24,  599,  238,  316,   20,    1, 4352,  119,   28,
        285,  180,  186,  133, 2962,  153,  604,    1,  615,  129,   16,
          1, 2150, 1394,   20,   48,  733,    5,   26,   13,   78,    7,
          7, 7598,   81, 5663,   47,   74,    3, 1897,  146,    9,   58,
        932,   78,    5,   63,    3, 1044,    4,   11,   16,   24,  692,
       1520,  145,  247,    1, 2567,    8,    1,  615,  129,   13,   24,
        199,  520,    7,    7,  789, 2011,   17,  103,    9,    2,  354,
         54, 1293, 4761,  590])

In [33]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [34]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [35]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [36]:
x_train_text[1]

"The acting is pretty cheesy, but for the people in this area up in the 80s and are now Detroit area automotive engineers, this is a great movie. I even work with a Japanese supplier so that makes this movie even more funny.<br /><br />Jay Leno was showing his age last night on The Tonight Show! He looks pretty young here...17 years ago. The opening scene, with the drag race on what appears to be Woodward Ave was great.<br /><br />Leno also owns some bad a** cars now, it would e great to see a remake of this with his modern collection. I'm sure the blown Vette in the opening scene was his own car.<br /><br />Typical 80s movie. Watch it and enjoy. No computer generated crap!"

In [37]:
tokens_to_string(x_train_tokens[1])

"the acting is pretty cheesy but for the people in this area up in the 80s and are now detroit area this is a great movie i even work with a japanese so that makes this movie even more funny br br jay leno was showing his age last night on the tonight show he looks pretty young here 17 years ago the opening scene with the drag race on what appears to be was great br br leno also owns some bad a cars now it would e great to see a remake of this with his modern collection i'm sure the blown in the opening scene was his own car br br typical 80s movie watch it and enjoy no computer generated crap"

In [85]:
model = Sequential()

In [86]:
embedding_size = 8

In [87]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [88]:
model.add(GRU(units=16, return_sequences=True, dropout=0.2 ))

In [89]:
model.add(GRU(units=8, return_sequences=True, dropout=0.2 ))

In [90]:
model.add(GRU(units=4))

In [91]:
model.add(Dense(1, activation='sigmoid'))

In [92]:
optimizer = Adam(lr=1e-3)

In [93]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [94]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_20 (GRU)                 (None, None, 16)          1200      
_________________________________________________________________
gru_21 (GRU)                 (None, None, 8)           600       
_________________________________________________________________
gru_22 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


In [95]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=128)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 238s - loss: 0.6912 - acc: 0.5292 - val_loss: 0.7566 - val_acc: 0.0168
Epoch 2/3
23750/23750 [==============================] - 258s - loss: 0.6841 - acc: 0.5487 - val_loss: 0.6877 - val_acc: 0.5280
Epoch 3/3
23750/23750 [==============================] - 264s - loss: 0.6652 - acc: 0.5934 - val_loss: 0.6412 - val_acc: 0.6464
CPU times: user 39min 48s, sys: 4min 6s, total: 43min 54s
Wall time: 12min 43s


In [97]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=128)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 262s - loss: 0.6392 - acc: 0.6313 - val_loss: 0.6141 - val_acc: 0.6688
Epoch 2/3
23750/23750 [==============================] - 260s - loss: 0.5819 - acc: 0.6897 - val_loss: 0.5657 - val_acc: 0.7112
Epoch 3/3
23750/23750 [==============================] - 263s - loss: 0.4828 - acc: 0.7718 - val_loss: 0.4164 - val_acc: 0.7984
CPU times: user 41min 20s, sys: 4min 24s, total: 45min 44s
Wall time: 13min 6s


In [98]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=10, batch_size=128)

Train on 23750 samples, validate on 1250 samples
Epoch 1/10
23750/23750 [==============================] - 261s - loss: 0.4374 - acc: 0.7995 - val_loss: 0.5002 - val_acc: 0.7552
Epoch 2/10
23750/23750 [==============================] - 260s - loss: 0.4065 - acc: 0.8201 - val_loss: 0.4372 - val_acc: 0.7920
Epoch 3/10
23750/23750 [==============================] - 262s - loss: 0.3791 - acc: 0.8347 - val_loss: 0.3878 - val_acc: 0.8256
Epoch 4/10
23750/23750 [==============================] - 261s - loss: 0.3585 - acc: 0.8472 - val_loss: 0.3756 - val_acc: 0.8272
Epoch 5/10
23750/23750 [==============================] - 260s - loss: 0.3363 - acc: 0.8575 - val_loss: 0.5386 - val_acc: 0.7576
Epoch 6/10
23750/23750 [==============================] - 256s - loss: 0.3178 - acc: 0.8670 - val_loss: 0.3411 - val_acc: 0.8488
Epoch 7/10
23750/23750 [==============================] - 258s - loss: 0.3009 - acc: 0.8752 - val_loss: 0.2898 - val_acc: 0.8744
Epoch 8/10
23750/23750 [========================

In [99]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=128)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 263s - loss: 0.2481 - acc: 0.9025 - val_loss: 0.3507 - val_acc: 0.8568
Epoch 2/3
23750/23750 [==============================] - 260s - loss: 0.2353 - acc: 0.9084 - val_loss: 0.2939 - val_acc: 0.8784
Epoch 3/3
23750/23750 [==============================] - 263s - loss: 0.2245 - acc: 0.9125 - val_loss: 0.3277 - val_acc: 0.8688
CPU times: user 41min 32s, sys: 4min 24s, total: 45min 56s
Wall time: 13min 6s


In [96]:
??model.evaluate

In [100]:
%%time
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================] - 163s   
CPU times: user 7min 44s, sys: 1min 5s, total: 8min 50s
Wall time: 2min 43s


In [101]:
from tensorflow.python.keras.models import load_model
model.save('NLP-test.h5')

In [102]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 85.78%


In [103]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

CPU times: user 20.7 s, sys: 2.22 s, total: 22.9 s
Wall time: 7.34 s


In [104]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [105]:
cls_true = np.array(y_test[0:1000])

In [106]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [107]:
len(incorrect)

148

In [108]:
idx = incorrect[0]
idx

1

In [109]:
text = x_test_text[idx]
text

'This is a beautiful, funny, vivid film. It\'s even better than "Nuovo Cinema Paradiso" -- which it parallels but doesn\'t replicate. The story completes a full circle and had the theater beaming as the credits rolled. A hundred years after this story takes place, we\'re just as intrigued by flickering images in a dark theater.'

In [110]:
y_pred[idx]

0.25336725

In [111]:
cls_true[idx]

1.0

## New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

In [184]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Yuck nasty!!! Horrible scenes. Nothing like the first movie which was awesome. Why add yuck when it could have been a fun family movie..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Ok date night movie. I still think the version with Robin Williams is the best. Very creative to modernize this one with the video game approach."
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [185]:
tokens = tokenizer.texts_to_sequences(texts)

To input texts with different lengths into the model, we also need to pad and truncate them.

In [186]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 544)

We can now use the trained model to predict the sentiment for these texts.

In [187]:
model.predict(tokens_pad)

array([[0.98008496],
       [0.98612565],
       [0.9631618 ],
       [0.19174291],
       [0.29266408],
       [0.9609741 ],
       [0.8180951 ],
       [0.05263658]], dtype=float32)

In [139]:
layer_embedding = model.get_layer('layer_embedding')

In [140]:
weights_embedding = layer_embedding.get_weights()[0]

In [141]:
weights_embedding.shape

(10000, 8)

In [142]:
token_good = tokenizer.word_index['good']
token_good

49

In [143]:
token_great = tokenizer.word_index['great']
token_great

78

In [144]:
weights_embedding[token_good]

array([0.88645536, 0.3461345 , 0.6933949 , 0.63914216, 0.9387442 ,
       0.01676616, 0.20363094, 1.0874246 ], dtype=float32)

In [145]:
weights_embedding[token_great]

array([ 1.4496535 ,  0.9262606 ,  0.43448642,  0.6957154 ,  0.5913994 ,
        0.23221469, -0.3196215 ,  0.05388168], dtype=float32)

Similarly, we can compare the embeddings for the words 'bad' and 'horrible'.

In [146]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [147]:
weights_embedding[token_bad]

array([-0.2988648 ,  0.3990402 , -0.04943583,  0.9046605 , -0.3187948 ,
        0.1623765 ,  0.17520568,  0.38988644], dtype=float32)

In [148]:
weights_embedding[token_horrible]

array([ 0.11061659, -0.65782636, -0.06191707,  1.1573843 , -0.071692  ,
        0.6390802 ,  0.984568  ,  0.16024065], dtype=float32)

In [149]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

In [150]:
print_sorted_words('great', metric='cosine')

Distance from 'great':
0.000 - great
0.023 - wonderful
0.029 - moved
0.029 - intensity
0.035 - comments
0.035 - touched
0.037 - queens
0.040 - gives
0.041 - smooth
0.041 - greatest
...
1.124 - ripped
1.127 - quasi
1.133 - mess
1.134 - mst3k
1.160 - is
1.162 - unwatchable
1.165 - insult
1.191 - pointless
1.196 - pathetic
1.203 - overlong


In [151]:
print_sorted_words('worst', metric='cosine')

Distance from 'worst':
0.000 - worst
0.034 - barrel
0.037 - fails
0.043 - awful
0.050 - drivel
0.053 - inept
0.054 - paid
0.066 - save
0.075 - producers
0.077 - education
...
1.500 - extremely
1.507 - jacques
1.515 - approaches
1.521 - suits
1.534 - is
1.547 - marvelous
1.585 - nevertheless
1.607 - 10
1.613 - definitely
1.655 - very
